In [25]:
import pandas as pd
from nyaggle.ensemble import stacking
from thunderbolt import Thunderbolt
from sklearn.metrics import roc_auc_score

In [7]:
tb = Thunderbolt("./resources")
tb.get_task_df(all_data=True).query('task_name == "Predict"')

,task_id,task_name,last_modified,task_params,task_hash,task_log
29,29,Predict,2020-02-24 11:10:54.270416,{'random_state': '110'},cef1f7df9ebe0382b7fd1c5626eb75f8,{'file_path': ['./resources/./submit/lgb_110/o...
96,96,Predict,2020-02-25 09:49:03.728846,{'random_state': '110'},b7904f20343c92d87dea6cb66f5c3bed,{'file_path': ['./resources/./submit/lgb_110/o...


In [10]:
lgb_oof = pd.read_csv(
    "./resources/submit/lgb_110/oof_b7904f20343c92d87dea6cb66f5c3bed.csv"
)
lgb_sub = pd.read_csv(
    "./resources/submit/lgb_110/submit_b7904f20343c92d87dea6cb66f5c3bed.csv"
)

In [13]:
lgb_oof.head()

,id,target
0,0,0.041756
1,1,0.118460
2,2,0.274174
3,3,0.034475
4,4,0.114848


In [12]:
nn_oof = pd.read_csv("./public_kernels/NN/oof.csv")
nn_sub = pd.read_csv("./public_kernels/NN/dnn_cv_submission.csv")

In [14]:
nn_oof

,id,dnn_oof
0,0,0.072350
1,1,0.119191
2,2,0.319075
3,3,0.021030
4,4,0.190107
...,...,...
599995,599995,0.007340
599996,599996,0.231588
599997,599997,0.111288
599998,599998,0.219415


In [16]:
y = pd.read_csv('../input/cat-in-the-dat-ii/train.csv', usecols=['target'])
y.head()

,target
0,0
1,0
2,0
3,0
4,0


In [26]:
result = stacking(
    test_predictions=[lgb_sub["target"].values, nn_sub["target"].values],
    oof_predictions=[lgb_oof["target"].values, nn_oof["dnn_oof"].values],
    y=y['target'],
    eval_func=roc_auc_score
)

/home/yuto/cat-in-the-dat-ii/.venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/home/yuto/cat-in-the-dat-ii/.venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/home/yuto/cat-in-the-dat-ii/.venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  F

In [27]:
result

EnsembleResult(test_prediction=array([0.11482836, 0.21283407, 0.14264451, ..., 0.5374129 , 0.21363856,
       0.20045877]), oof_prediction=array([0.07359298, 0.09900304, 0.25839367, ..., 0.09485324, 0.17317049,
       0.07501474]), score=0.7940175874031385)

In [30]:
stacking_sub = pd.read_csv('../input/cat-in-the-dat-ii/sample_submission.csv', index_col="id")
stacking_sub['target'] = result.test_prediction
stacking_sub.head()

,target
id,
600000,0.114828
600001,0.212834
600002,0.142645
600003,0.110216
600004,0.116851


In [31]:
stacking_sub.to_csv('./stacking_result/lgb_nn.csv')